# Alexander Invariants for MOY-graphs
----------
### by Egor Alimpiev

## Definitions

In [4]:
class X():
    
    def __init__(self, elist):
        # read in a crossing as a list of 4 edges
        assert elist[0][1] == elist[2][1] and elist[1][1] == elist[3][1]
        self.crossing = list(elist)
    
    @property
    def i(self):
        return self.crossing[0]
    
    @property
    def j(self):
        return self.crossing[1]
    
    @property
    def k(self):
        return self.crossing[2]
    
    @property
    def l(self):
        return self.crossing[3]
    
    @property
    def type(self):
        return 'x'
    
    @property
    def seg_ids(self):
        return [s[0] for s in self.crossing]
    
    
    @property
    def sign(self):
        if self.crossing[1][0] - self.crossing[3][0] == 1 or self.crossing[3][0] - self.crossing[1][0] > 1:
            return 1
        elif self.crossing[3][0] - self.crossing[1][0] == 1 or self.crossing[1][0] - self.crossing[3][0] > 1:
            return -1
        
    def __repr__(self):
        return 'Crossing '+str([i[0] for i in self.crossing])

In [5]:
class V():
    
    def __init__(self, inc, out):
        self.incoming_edges = inc
        self.outgoing_edges = out
        self.inc_len = len(inc)
        self.out_len = len(out)
        
    @property
    def incoming(self):
        return self.incoming_edges
    
    @property
    def outgoing(self):
        return self.outgoing_edges
    
    @property
    def n_incoming(self):
        return len(self.incoming_edges)
    
    @property
    def n_outgoing(self):
        return len(self.outgoing_edges)
    
    @property
    def in_seg_ids(self):
        return [s[0] for s in self.incoming_edges]
    
    @property
    def out_seg_ids(self):
        return [s[0] for s in self.outgoing_edges]
    
    @property
    def type(self):
        return 'v'
    
    def __repr__(self):
        return 'Vertex with ' + str([i[0] for i in self.incoming_edges]) \
             + ' incoming and '+ str([i[0] for i in self.outgoing_edges]) + ' outgoing.'

In [6]:
class MOY():
    def __init__(self, xvs, ):
        self.t = var('t')
        self.abs_delta = 0
        
        # store the initial list of crossings and vertices
        self.xvs = xvs  
        
        #separate it into crossings and vertices
        self.xs = [i for i in xvs if i.type == 'x']
        self.vs = [i for i in xvs if i.type == 'v']
        
        # we get segments as inputs, so we want to go from segments to actual edges and vice versa
        max_seg_id = -1
        for x in self.xs:
            max_seg_id = max(max_seg_id, max(x.seg_ids))
        for v in self.vs:
            max_seg_id = max(max_seg_id, max(v.in_seg_ids), max(v.out_seg_ids))
        self.n_seg = max_seg_id + 1
        
        # this is a standard union-find implementation to find the maps from segments to edges and back
        self.segment_parents = [i for i in range(self.n_seg)]
        
        for x in self.xs:
            self._union(self.segment_parents, x.i[0], x.k[0])
            self._union(self.segment_parents, x.j[0], x.l[0])
        
        # this list gives an (arbitrary) index of an edge for each segment
        self.segment_parents = [self._find(self.segment_parents, i) for i in range(self.n_seg)]
        
        # self.edges stores the indices of segments that represent edges, as chosed by union-find
        self.n_edges = len(set(self.segment_parents))
        self.edges = list(set(self.segment_parents))
        
        self.edge_to_seg = {i:[j for j in range(self.n_seg) if self._find(self.segment_parents, j) == i] for i in self.edges}
        self.seg_to_edge = self.segment_parents
        
        
        # this is a straightforward way to record a color of each segment
        self.seg_to_col = {}
        for x in self.xs:
            for s in x.crossing:
                if s[0] not in self.seg_to_col:
                    self.seg_to_col[s[0]] = s[1]
        
        for v in self.vs:
            for s in v.incoming:
                if s[0] not in self.seg_to_col:
                    self.seg_to_col[s[0]] = s[1]
            for s in v.outgoing:
                if s[0] not in self.seg_to_col:
                    self.seg_to_col[s[0]] = s[1]
        
        # in case we ever need this, we also record a color for each edge in a separate list
        self.edge_to_col = {e:self.seg_to_col[self.edge_to_seg[e][0]] for e in self.edges}
                    
        # |Cr(D)| in the paper
        self.n_crossings = len(self.xs) + sum([v.n_incoming for v in self.vs])
        
        # under the assumption of a connected graph diagram, we get |Re(D)| as follows
        self.n_regions = self.n_crossings + 2
        
        # now, we need to somehow store which segments are adjacent to which regions
        # so, we first have 2*n_segments 'candidate regions' -- on the left and right of each segment
        # then, for each crossing we unite all we can:
        # for example, in positive X(i,j,k,l) we unite 
        #    - 'right of i' and 'right of j'
        #    - 'left of j'  and 'right of k'
        #    - 'left of k'  and 'left of l'
        #    - 'right of l' and 'left of i'
        # in negative X(i,j,k,l) we unite 
        #    - 'right of i' and 'left of j'
        #    - 'right of j' and 'right of k'
        #    - 'left of k'  and 'right of l'
        #    - 'left of l'  and 'left of i'
        
        # for each vertex, we first do not want to introduce the circle regions
        # we take care about them later, since we would know which other two regions are adjacent to them!
        # so, we just unite left of first incoming with left of first outgoing, etc.
        
        reg_c = [i for i in range(2*self.n_seg)]
        for x in self.xs:
            if x.sign == 1:
                # 2*x.i -- left; 2*x.i + 1 -- right
                self._union(reg_c, 2*x.i[0] + 1, 2*x.j[0] + 1)
                self._union(reg_c, 2*x.j[0],     2*x.k[0] + 1)
                self._union(reg_c, 2*x.k[0],     2*x.l[0])
                self._union(reg_c, 2*x.l[0] + 1, 2*x.i[0])
            elif x.sign == -1:
                self._union(reg_c, 2*x.i[0] + 1, 2*x.j[0])
                self._union(reg_c, 2*x.j[0] + 1, 2*x.k[0] + 1)
                self._union(reg_c, 2*x.k[0],     2*x.l[0] + 1)
                self._union(reg_c, 2*x.l[0],     2*x.i[0])
            else: 
                print('something is really wrong!')
                
        for v in self.vs:
            self._union(reg_c, 2*v.incoming[0][0], 2*v.outgoing[-1][0])
            self._union(reg_c, 2*v.incoming[-1][0]+1, 2*v.outgoing[0][0]+1)
            if v.n_incoming > 1:
                for i in range(v.n_incoming - 1):
                    self._union(reg_c, 2*v.incoming[i][0]+1, 2*v.incoming[i+1][0])
            if v.n_outgoing > 1:
                for i in range(v.n_outgoing - 1):
                    self._union(reg_c, 2*v.outgoing[i][0], 2*v.outgoing[i+1][0]+ 1)
                    
        self.reg_parents = [self._find(reg_c, i) for i in range(2*self.n_seg)]
        
        # seg to region in the form (r_left, r_right), i. e. for each segment we record 
        # a region on the left and on the right
        self.seg_to_reg = {i:[self.reg_parents[2*i], self.reg_parents[2*i+1]] for i in range(self.n_seg)}
        
        # this list stores representatives of noncircular regions as with edges above
        self.n_noncirc_regions = len(set(self.reg_parents))
        self.noncirc_regions = list(set(self.reg_parents))
        
        # for each region we store the segments that bound it
        self.reg_to_seg = {i:[j//2 for j, r in enumerate(self.reg_parents) if r == i] for i in self.noncirc_regions}
        
        # for each region we store the crossings and vertices adjacent to it
        # we encode it by putting a tuple: an object itself plus some additional info.
        # for crossings:  position of a region
        # for vertices: whether the region is adjacent to incoming edges in case we need it
        self.reg_to_xvs = {i:[] for i in self.noncirc_regions}
        for x in self.xs:
            if x.sign == 1:
                self.reg_to_xvs[self.seg_to_reg[x.i[0]][0]].append((x,'S'))
                self.reg_to_xvs[self.seg_to_reg[x.i[0]][1]].append((x,'E'))
                self.reg_to_xvs[self.seg_to_reg[x.k[0]][0]].append((x,'W'))
                self.reg_to_xvs[self.seg_to_reg[x.k[0]][1]].append((x,'N'))
            elif x.sign == -1:
                self.reg_to_xvs[self.seg_to_reg[x.i[0]][0]].append((x,'W'))
                self.reg_to_xvs[self.seg_to_reg[x.i[0]][1]].append((x,'S'))
                self.reg_to_xvs[self.seg_to_reg[x.k[0]][0]].append((x,'N'))
                self.reg_to_xvs[self.seg_to_reg[x.k[0]][1]].append((x,'E'))
        for  v in self.vs:
            for s, _ in v.incoming:
                self.reg_to_xvs[self.seg_to_reg[s][0]].append((v,'inc'))
            self.reg_to_xvs[self.seg_to_reg[v.incoming[-1][0]][1]].append((v,'inc'))
            
            if len(v.outgoing) > 1:
                for i in range(len(v.outgoing)-1):
                    s, c = v.outgoing[i]
                    self.reg_to_xvs[self.seg_to_reg[s][0]].append((v,'not inc'))
        
        self.A = self.get_alexander_matrix()
        
    # just a check
    def is_MOY(self):
        return all([sum([c for s, c in v.outgoing]) == sum([c for s, c in v.outgoing]) for v in self.vs])
    
    def get_alexander_matrix(self):
        # get the dimensions of a matrix
        nrows = self.n_crossings
        ncols = self.n_regions
        
        # use sage to define a symbolic matrix of zeros
        A = matrix(SR, nrows, ncols)
        
        # first we iterate over the crossings, filling out all of the related entries of the matrix
        # (it is more convenient to make the outer loop go over regions)
        for j, (reg, adj_xvs) in enumerate(self.reg_to_xvs.items()):
            # this is a helper array to check if we have an adjacency
            adj_xvs_elems_only = [e[0] for e in adj_xvs]
            for i, x in enumerate(self.xs):
                if x in adj_xvs_elems_only:
                    contrib = 0
                    # this covers the possibility of the same crossing being present multiple times 
                    # for the same region 
                    x_present = [e for e in adj_xvs if e[0] == x]
                    for x, pos in x_present:
                        x_type = '+' if x.sign == 1 else '-'
                        # extract the color of a proper edge
                        col = self.seg_to_col[x.j[0]]
                        contrib += self.m_Cp(x_type, pos)*self.A_Cp(x_type, pos, col)
                    A[i,j] += contrib
        # ^ this way we have filled out the first len(xs) rows, corresponding to the crossings 
        # that do not arise from vertices. Next, i switch and iterate over the remaining crossings, adding contributions to each row
    
        # is still the row index that i will increment
        i = len(self.xs)
        # k serves to keep track of the circle region
        for k, v in enumerate(self.vs):
            # for each incoming edge we get a crossing
            for s, col in v.incoming:
                left_region = list(self.reg_to_xvs.keys()).index(self.seg_to_reg[s][0])
                circ_region = k + len(self.reg_to_xvs.keys())
                right_region = list(self.reg_to_xvs.keys()).index(self.seg_to_reg[s][1])

                A[i,left_region] += self.m_Cp('circ', 'left')*self.A_Cp('circ', 'left', col)
                A[i,circ_region] += self.m_Cp('circ', 'top')*self.A_Cp('circ', 'top', col)
                A[i,right_region] += self.m_Cp('circ', 'right')*self.A_Cp('circ', 'right', col)
                i += 1
        return A
    
    def get_state_sum(self, delta_segment = 0, left_region_index = 0):
        # first create a delta factor, as described in Definition 2.10
        n = left_region_index
        self.abs_delta = self.t**(n - self.seg_to_col[delta_segment]) - self.t**(n)  
        
        # now delete the two columns and compute the deretminant
        R_u, R_v = self.seg_to_reg[delta_segment]
        square_A = self.A[:,[i for i in range(self.n_regions) if i != R_u and i != R_v]]
        

        return square_A.det()/self.abs_delta
    
    def get_alexander_invariant(self, delta_segment = 0, left_region_index = 0):
        ss = self.get_state_sum(delta_segment, left_region_index)
        denum = (self.t**(-1/2)- self.t**(1/2))**(len(self.vs) - 1)
        return ss/denum
                
    # get the m_Cp contribution of a vertex to a region
    # given the type of vertex and the relative location of the region
    def m_Cp(self, xv_type, pos):   
        if xv_type == 'circ':
            if pos == 'top':
                return 1
            elif pos == 'left':
                return -1
            elif pos == 'right':
                return 1
        elif xv_type == '+':
            if pos == 'N':
                return 1
            elif pos == 'S':
                return 1
            elif pos == 'W':
                return -1
            elif pos == 'E':
                return -1
        elif xv_type == '-':
            if pos == 'N':
                return -1
            elif pos == 'S':
                return -1
            elif pos == 'W':
                return 1
            elif pos == 'E':
                return 1
            
    # same for the other contribution A_Cp
    def A_Cp(self, xv_type, pos, color):
        if xv_type == 'circ':
            if pos == 'top':
                return self.t**(-color/2) - self.t**(color/2)
            elif pos == 'left':
                return self.t**(-color/2)
            elif pos == 'right':
                return self.t**(color/2)
        elif xv_type == '+':
            if pos == 'N':
                return self.t**(-color)
            elif pos == 'S':
                return 1
            elif pos == 'W':
                return self.t**(-color)
            elif pos == 'E':
                return 1
        elif xv_type == '-':
            if pos == 'N':
                return self.t**(color)
            elif pos == 'S':
                return 1
            elif pos == 'W':
                return 1
            elif pos == 'E':
                return self.t**(color)
        
    def _find(self, l, seg):
        if seg == l[seg]:
            return seg
        l[seg] = self._find(l, l[seg])
        return l[seg]
        
    def _union(self, l, seg_i, seg_j):
        iloc = self._find(l, seg_i)
        jloc = self._find(l, seg_j)
        
        if iloc <= jloc:
            l[jloc] = iloc
        else:
            l[iloc] = jloc
        return 0

## Usage Instructions and examples.

Here we implement the construction of the paper *"AN ALEXANDER POLYNOMIAL FOR MOY GRAPHS"* (https://arxiv.org/pdf/1708.09092.pdf).

The usage is quite simple. 
The input is a MOY-graph, given as a list of vertices and crossings in the following way:

1 - On a given graph diagram number every edge segment (separated by either a vertex or a crossing) with an integer starting with zero, *taking care to number all consecutive segments of each edge by consecutive numbers* (needed to correctly determine the sign of each crossing).

2 - Each segment is defined by a tuple `(segment_number, color)`, where, of course, all segments belonging to the same edge share the same color.

3 - Each crossing is defined as an object `X([i,j,k,l])` as in the usual PD-code notation for knots, where `i, j, k, l` are segments (edges) listed counterclockwise starting with lower incoming one.

4 - Each vertex is defined as an object `V([i1, i2, ...], [o1, o2, ...])` where the first argument contains incoming segments (edges), *listed from leftmost to the rightmost*, and the second argument contains outgoing segments (edges), *listed from rightmost to leftmost*, so that overall we list all segments counterclockwise.

5 - Finally, a graph diagram is defined as an object `MOY([x1, ..., v1, ...])` where the argument is a list of all crossings and vertices in no particular order.

### Example 1

The first example is a graph diagram from Figure 4 of the paper, with a particular choice of colors.

![](imgs/MOY_ex_1.png)

We record it according to the algorithm above.

In [263]:
# define the crossings and vertices
X1 = X([ (3,1), (0,2), (4,1), (1,2) ])
V1 = V( [ (1,2), (4,1) ], [ (2,3) ])
V2 = V( [ (2,3) ], [ (0,2), (3,1) ])

# define the graph diagram
example= MOY([X1, V1, V2])

`MOY.A` outputs the Alexander matrix as described in Definition 2.14 of the paper.

In [264]:
latex(example.A)

\left(\begin{array}{rrrrrr}
-1 & t^{2} & 1 & -t^{2} & 0 & 0 \\
0 & 0 & -\frac{1}{t} & t & -t + \frac{1}{t} & 0 \\
0 & \sqrt{t} & 0 & -\frac{1}{\sqrt{t}} & -\sqrt{t} + \frac{1}{\sqrt{t}} & 0 \\
0 & t^{\frac{3}{2}} & -\frac{1}{t^{\frac{3}{2}}} & 0 & 0 & -t^{\frac{3}{2}} + \frac{1}{t^{\frac{3}{2}}}
\end{array}\right)

$$
\left(\begin{array}{rrrrrr}
-1 & t^{2} & 1 & -t^{2} & 0 & 0 \\
0 & 0 & -\frac{1}{t} & t & -t + \frac{1}{t} & 0 \\
0 & \sqrt{t} & 0 & -\frac{1}{\sqrt{t}} & -\sqrt{t} + \frac{1}{\sqrt{t}} & 0 \\
0 & t^{\frac{3}{2}} & -\frac{1}{t^{\frac{3}{2}}} & 0 & 0 & -t^{\frac{3}{2}} + \frac{1}{t^{\frac{3}{2}}}
\end{array}\right)
$$

Taking a determinant after dropping columns corresponding to two adjacent regions, we get a state sum $\langle D, c \rangle$.

In [215]:
example.get_state_sum(delta_segment = 0, left_region_index=0)

(t^2 - 1/t + 1/t^3 - 1)/(1/t^2 - 1)

In [216]:
example.get_alexander_invariant(delta_segment = 0, left_region_index=0)

-(t^2 - 1/t + 1/t^3 - 1)/((sqrt(t) - 1/sqrt(t))*(1/t^2 - 1))

In [273]:
a_inv = example.get_alexander_invariant(delta_segment = 0, left_region_index=0)
a_inv.full_simplify()

(t^2 + t + 1)/sqrt(t)

### Example 2

Defining a graph diagram with an arbitrary balanced coloring works as well. One only needs to take care of defining color variables as sage object.

![](imgs/MOY_ex_2.png)

In [224]:
i = var('i')
j = var('j')
ex_xvs2 = [ X([ (3,j), (0,i), (4,j), (1,i) ]), 
            V([ (1,i), (4,j) ], [ (2,i+j) ]), 
            V([ (2,i+j) ], [ (0,i), (3,j) ]), 
          ]
example2 = MOY(ex_xvs2)

In [180]:
latex(example2.A)

\left(\begin{array}{rrrrrr}
-1 & t^{i} & 1 & -t^{i} & 0 & 0 \\
0 & 0 & -\frac{1}{t^{\frac{1}{2} \, i}} & t^{\frac{1}{2} \, i} & -t^{\frac{1}{2} \, i} + \frac{1}{t^{\frac{1}{2} \, i}} & 0 \\
0 & t^{\frac{1}{2} \, j} & 0 & -\frac{1}{t^{\frac{1}{2} \, j}} & -t^{\frac{1}{2} \, j} + \frac{1}{t^{\frac{1}{2} \, j}} & 0 \\
0 & t^{\frac{1}{2} \, i + \frac{1}{2} \, j} & -t^{-\frac{1}{2} \, i - \frac{1}{2} \, j} & 0 & 0 & -t^{\frac{1}{2} \, i + \frac{1}{2} \, j} + t^{-\frac{1}{2} \, i - \frac{1}{2} \, j}
\end{array}\right)

This is the matrix at the bottom of page 10 of the paper, up to permutation of rows and columns.

$$
\left(\begin{array}{rrrrrr}
-1 & t^{i} & 1 & -t^{i} & 0 & 0 \\
0 & 0 & -t^{-\frac{i}{2} } & t^{\frac{i}{2} } & -t^{\frac{i}{2} } + t^{-\frac{i}{2} } & 0 \\
0 & t^{\frac{j}{2}  } & 0 & -t^{-\frac{j}{2} } & -t^{\frac{j}{2} } + t^{-\frac{j}{2} } & 0 \\
0 & t^{\frac{i}{2}  + \frac{j}{2}} & -t^{-\frac{i}{2}  - \frac{j}{2} } & 0 & 0 & -t^{\frac{i}{2}  + \frac{j}{2}  } + t^{-\frac{i}{2}  - \frac{j}{2} }
\end{array}\right)
$$

So, we try to get the state sum, and compare it to the expression given in the paper.

In [226]:
ss = example2.get_state_sum(delta_segment = 0, left_region_index=0)
latex(ss)

\frac{t^{i} - \frac{1}{t^{j}} + \frac{1}{t^{i} t^{j}} - 1}{\frac{1}{t^{i}} - 1}

$$
\frac{t^{i} - \frac{1}{t^{j}} + \frac{1}{t^{i} t^{j}} - 1}{\frac{1}{t^{i}} - 1}
$$

In [227]:
ratio = (-(t**((i+j)/2)-t**(-(i+j)/2))*t**((i-j)/2))/ss
ratio.full_simplify()

1

After simplification, we see that the expression in the paper (Example 2.13) and my result are the same.

In [272]:
example2.get_alexander_invariant(delta_segment = 0, left_region_index=0).full_simplify()

(t^(i + j) - 1)*t^(-j + 1/2)/(t - 1)

### Example 3

Let us take a singular knot as a next example.

![](imgs/MOY_ex_3.png)

In [265]:
X1 = X([(0,1),(4,1),(1,1),(3,1)])
X2 = X([(4,1),(2,1),(5,1),(1,1)])
V1 = V([(5,1),(2,1)],[(0,1),(3,1)])

example3 = MOY([X1,X2,V1])

example3.get_state_sum(delta_segment = 0, left_region_index=1) #note the value for the left region index

(1/t - 1)/(t - 1)

In [266]:
latex(example3.A)

\left(\begin{array}{rrrrrr}
1 & -1 & -\frac{1}{t} & \frac{1}{t} & 0 & 0 \\
0 & -1 & -\frac{1}{t} & 1 & \frac{1}{t} & 0 \\
0 & 0 & -\frac{1}{\sqrt{t}} & 0 & \sqrt{t} & -\sqrt{t} + \frac{1}{\sqrt{t}} \\
0 & \sqrt{t} & 0 & 0 & -\frac{1}{\sqrt{t}} & -\sqrt{t} + \frac{1}{\sqrt{t}}
\end{array}\right)

$$
\left(\begin{array}{rrrrrr}
1 & -1 & -\frac{1}{t} & \frac{1}{t} & 0 & 0 \\
0 & -1 & -\frac{1}{t} & 1 & \frac{1}{t} & 0 \\
0 & 0 & -\frac{1}{\sqrt{t}} & 0 & \sqrt{t} & -\sqrt{t} + \frac{1}{\sqrt{t}} \\
0 & \sqrt{t} & 0 & 0 & -\frac{1}{\sqrt{t}} & -\sqrt{t} + \frac{1}{\sqrt{t}}
\end{array}\right)
$$

In [271]:
example3.get_alexander_invariant(delta_segment = 0, left_region_index=1).full_simplify()

-1/t

### Example 4

Inputing a usual trefoil knot (no vertices), we see that up to some factor in the denominator, we get the usual Alexander polynomial of a knot.

In [268]:
X1 = X([(0,1),(4,1),(1,1),(3,1)])
X2 = X([(4,1),(2,1),(5,1),(1,1)])
X3 = X([(2,1),(0,1),(3,1),(5,1)])

example4 = MOY([X1,X2,X3])

example4.get_state_sum(delta_segment = 0, left_region_index=1) #note the value for the left region index

(1/t - 1/t^2 + 1/t^3)/(t - 1)

In [270]:
example4.get_alexander_invariant(delta_segment = 0, left_region_index=1).full_simplify()

-(t^2 - t + 1)/t^(7/2)

### Example 5

Next example is once again from the original paper.

![](imgs/MOY_ex_4.png)

In [243]:
i = var('i')
j = var('j')

# vertices
V1 = V([(12,i+j)],
       [(5,i),(0,j)])
V2 = V([(9,i), (4,j)],
       [(10,i+j)])

X1 = X([
    (0,j),
    (9,i),
    (1,j),
    (8,i)
])

# crossings
X2 = X([
    (7,i),
    (12,i+j),
    (8,i),
    (11,i+j)
])

X3 = X([
    (10,i+j),
    (2,j),
    (11,i+j),
    (1,j)
])

X4 = X([
    (2,j),
    (6,i),
    (3,j),
    (7,i)
])

X5 = X([
    (5,i),
    (3,j),
    (6,i),
    (4,j)
])

In [244]:
example5 = MOY([V1,V2,X1,X2,X3,X4,X5])

example5.get_alexander_invariant(delta_segment = 4, left_region_index=0).full_simplify() #note the value for the left region index

((t^(3*i) - t^(2*i))*t^(3*j) - (t^(3*i) + 2*t^(2*i) - 2*t^i)*t^(2*j) + (2*t^(2*i) - 1)*t^j - t^i + 1)*t^(-2*i - 2*j + 1/2)/(t - 1)

This expression is too big. Try $i = j = 1$.

In [245]:
i = 1
j = 1

# vertices
V1 = V([(12,i+j)],
       [(5,i),(0,j)])
V2 = V([(9,i), (4,j)],
       [(10,i+j)])

# crossings
X1 = X([
    (0,j),
    (9,i),
    (1,j),
    (8,i)
])

X2 = X([
    (7,i),
    (12,i+j),
    (8,i),
    (11,i+j)
])

X3 = X([
    (10,i+j),
    (2,j),
    (11,i+j),
    (1,j)
])

X4 = X([
    (2,j),
    (6,i),
    (3,j),
    (7,i)
])

X5 = X([
    (5,i),
    (3,j),
    (6,i),
    (4,j)
])

In [254]:
example5 = MOY([V1,V2,X1,X2,X3,X4,X5])

(example5.get_alexander_invariant(delta_segment = 4, left_region_index=0).full_simplify() \
                                                                     *(example5.t**(7/2))).factor()

(t^4 - 2*t^3 - t^2 + 2*t - 1)*(t + 1)

In [262]:
D = example5.get_alexander_invariant(delta_segment = 4, left_region_index=0)
D_inv = D.substitute(t = example5.t**(-1))
(D/D_inv).full_simplify() # != t^k => the graph is chiral!

-(t^4 - 2*t^3 - t^2 + 2*t - 1)/(t^6 - 2*t^5 + t^4 + 2*t^3 - t^2)

$$
\left(\begin{array}{rrrrrrrrrr}
1 & -1 & -\frac{1}{t^{j}} & \frac{1}{t^{j}} & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & -1 & -\frac{1}{t^{i}} & 1 & \frac{1}{t^{i}} & 0 & 0 & 0 & 0 & 0 \\
0 & -1 & 0 & 0 & 1 & -1 & 1 & 0 & 0 & 0 \\
\frac{1}{t^{j}} & 0 & -\frac{1}{t^{j}} & 0 & 1 & -1 & 0 & 0 & 0 & 0 \\
1 & 0 & 0 & 0 & 0 & -1 & t^{i} & -t^{i} & 0 & 0 \\
-t^{j} + 1 & 0 & 0 & 0 & 0 & 0 & t^{j} & -1 & 0 & 0 \\
-\frac{1}{t^{\frac{1}{2} \, i}} & 0 & 0 & 0 & 0 & 0 & t^{\frac{1}{2} \, i} & 0 & -t^{\frac{1}{2} \, i} + \frac{1}{t^{\frac{1}{2} \, i}} & 0 \\
1 & 0 & -1 & 0 & 0 & 0 & 0 & 0 & 0 & 0
\end{array}\right)
$$